# Section 5 : Multispectral Remote Sensing Data in Python
## Chapter 9 : Landsat Exercises

[![Foo](https://www.colorado.edu/brand/sites/default/files/styles/medium/public/page/boulder-one-line_0.png)](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/)

Pour ce dernier notebook dédié spécifiquement aux données Landsat, quelques exercices s'imposent pour vérifier que les notions ont bien été assimilées. De façon assez simpliste, les deux prochaines questions concernent le recadrage et l'affichage d'un cliché satellite de Cold Springs différent de ceux que l'on a pu voir par le passé. Ce notebook vise donc à importer, recadrer et afficher une carte de Cold Springs avant les incendies de 2016.

In [1]:
import os
from glob import glob

import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
from shapely.geometry import mapping
import numpy as np
import numpy.ma as ma
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep


# Download data and set working directory
data = et.data.get_data('cold-springs-fire')
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))
